In [32]:
import numpy
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
numpy.random.seed(10)

In [33]:
data = pd.read_csv("target.csv")

In [34]:
data[:2]

,customer_id,card_tenure,risk_score,num_promoted,avg_bal,geo_group,res_type,Unnamed: 7
0,SB0000057824,198.0,755,0,3818.0,E,NaN,NaN
1,SB0000030501,206.0,647,0,3357.0,SE,CN,NaN


In [35]:
data=data.drop(['Unnamed: 7'], axis=1)
data=data.drop(['customer_id'], axis=1)

In [36]:
data = data.dropna(how='any')

In [37]:
data['res_type']= data['res_type'].map({'CN':0, 'CO': 1, 'RE': 2, 'TO': 3,'SI': 4}).astype(int) 

In [38]:
data=pd.get_dummies(data=data,columns=['geo_group'])

In [39]:
data[:2]

,card_tenure,risk_score,num_promoted,avg_bal,res_type,geo_group_E,geo_group_N,geo_group_SE,geo_group_W
1,206.0,647,0,3357.0,0,0.0,0.0,1.0,0.0
2,107.0,629,0,3728.0,0,0.0,1.0,0.0,0.0


In [40]:
X = data.drop(['res_type'], axis=1).values.astype('float64')

In [41]:
Y = data['res_type'].values.astype('int64')

In [42]:
X = StandardScaler().fit_transform(X)
train_Features, test_Features, train_Label, test_Label = train_test_split(X, Y, test_size=0.2, random_state=True) 
len(train_Features)

78804

In [43]:
from keras.utils import np_utils
train_Label=np_utils.to_categorical(train_Label)
test_Label=np_utils.to_categorical(test_Label)

In [44]:
train_Features.shape

(78804, 8)

In [45]:
train_Label.shape

(78804, 5)

In [46]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

In [47]:
model = Sequential()

In [48]:
model.add(Dense(units=24*6,   input_dim=8,
                kernel_initializer='uniform', 
                activation='relu'))

In [49]:
model.add(Dense(units=24*5, 
                kernel_initializer='uniform', 
                activation='relu'))

In [50]:
model.add(Dense(units=24*4, 
                kernel_initializer='uniform', 
                activation='relu'))

In [51]:
model.add(Dense(units=24*3, 
                kernel_initializer='uniform', 
                activation='relu'))

In [52]:
model.add(Dense(units=24*2, 
                kernel_initializer='uniform', 
                activation='relu'))

In [53]:
model.add(Dense(units=24, 
                kernel_initializer='uniform', 
                activation='relu'))

In [54]:
model.add(Dense(units=5, 
                kernel_initializer='uniform',
                activation='softmax'))

In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 144)               1296      
_________________________________________________________________
dense_9 (Dense)              (None, 120)               17400     
_________________________________________________________________
dense_10 (Dense)             (None, 96)                11616     
_________________________________________________________________
dense_11 (Dense)             (None, 72)                6984      
_________________________________________________________________
dense_12 (Dense)             (None, 48)                3504      
_________________________________________________________________
dense_13 (Dense)             (None, 24)                1176      
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 125       
Total para

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [ ]:
train_history =model.fit(x=train_Features, 
                         y=train_Label, 
                         validation_split=0.1, 
                         epochs=30, 
                         batch_size=800,verbose=1,callbacks=[early_stopping])

Train on 70923 samples, validate on 7881 samples
Epoch 1/30
70923/70923 [==============================] - 3s 40us/step - loss: 1.3451 - acc: 0.3854 - val_loss: 1.0056 - val_acc: 0.5499
Epoch 2/30
70923/70923 [==============================] - 3s 47us/step - loss: 0.7904 - acc: 0.6050 - val_loss: 0.6517 - val_acc: 0.7235
Epoch 3/30
70923/70923 [==============================] - 4s 61us/step - loss: 0.2613 - acc: 0.8943 - val_loss: 0.0581 - val_acc: 0.9802
Epoch 4/30
70923/70923 [==============================] - 2s 33us/step - loss: 0.0491 - acc: 0.9834 - val_loss: 0.0456 - val_acc: 0.9796
Epoch 5/30
70923/70923 [==============================] - 4s 54us/step - loss: 0.0498 - acc: 0.9787 - val_loss: 0.0609 - val_acc: 0.9699
Epoch 6/30
70400/70923 [============================>.] - ETA: 0s - loss: 0.0389 - acc: 0.9842

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
show_train_history(train_history,'acc','val_acc')

In [ ]:
show_train_history(train_history,'loss','val_loss')

In [ ]:
scores = model.evaluate(x=test_Features, 
                        y=test_Label)

In [ ]:
scores[1]